In [1]:
import time
import copy
import numpy as np
from skimage.filters import gaussian
import cv2 

import matplotlib.pyplot as plt
import pylab as pl



%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (15, 10)

from tifffile import imread, imsave
import pickle
import bobsegutils as bsu

from skimage.draw import circle

import csv

from skimage import io

import math

Populating the interactive namespace from numpy and matplotlib


/Users/prakash/anaconda/lib/python3.6/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['copy']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
### Read the results csv file, delete all entries which are not coordinates and write it as a new csv file
for i in range(1,20,1):
    #with open("/Users/prakash/Desktop/BobSegDataAndResults/FlowQuantification/time"+str(i)+"-"+str(i+1)+"/time_00"+str(i+1)+".csv","r") as source:
    with open("/Users/prakash/Desktop/BobSegDataAndResults/FlowQuantificationDataset2/time"+str(i)+"-"+str(i+1)+"/time_00"+str(i+1)+".csv","r") as source:
        rdr= csv.reader( source )
        with open("/Users/prakash/Desktop/BobSegDataAndResults/FlowQuantificationDataset2/time"+str(i)+"-"+str(i+1)+"/"+"Resultstime"+str(i)+"-"+str(i+1)+".csv","w") as result:
        #with open("/Users/prakash/Desktop/BobSegDataAndResults/FlowQuantification/time"+str(i)+"-"+str(i+1)+"/"+"Resultstime"+str(i)+"-"+str(i+1)+".csv","w") as result:
            wtr= csv.writer( result )
            for r in rdr:
                wtr.writerow( (r[5], r[6]) )

### The manually tracked coordinates from Fiji are processed before plotting

In [3]:
def load_file(filename):
    '''Make a list of tuples from a csv file
    '''
    fp = open(filename, 'Ur')
    data_list = []
    for line in fp:
        data_list.append(tuple((line.strip().split(','))))
    fp.close()
    return data_list

In [4]:
# The above created list elememts are string, we convert them to float below
center_points = []
manual_track_points = []
for f in range(1,20,1):
    #data_list = load_file("/Users/prakash/Desktop/BobSegDataAndResults/FlowQuantification/time"+str(f)+"-"+str(f+1)+"/Resultstime"+str(f)+"-"+str(f+1)+".csv")
    data_list = load_file("/Users/prakash/Desktop/BobSegDataAndResults/FlowQuantificationDataset2/time"+str(f)+"-"+str(f+1)+"/Resultstime"+str(f)+"-"+str(f+1)+".csv")
    datal = data_list[1:] # Ignoring the header row (first row of csv file)
    half = int(len(datal)/2)
    string_first_half = datal[0:half]
    string_second_half = datal[half:]
    float_first_half_list = []
    float_second_half_list = []
    for element in range(len(string_first_half)):
        number_element_first_half = []
        number_element_second_half = []
        number_element_first_half.append((float(string_first_half[element][0])))
        number_element_first_half.append((float(string_first_half[element][1])))
        float_first_half_list.append(tuple(number_element_first_half))
        number_element_second_half.append((float(string_second_half[element][0])))
        number_element_second_half.append((float(string_second_half[element][1])))
        float_second_half_list.append(tuple(number_element_second_half))
    center_points.append(float_first_half_list)
    manual_track_points.append(float_second_half_list)

/Users/prakash/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 'U' mode is deprecated
  after removing the cwd from sys.path.


The cell below makes a flat list out of maunal_track_points and center_points

In [5]:
center_points_flat_list = [item for sublist in center_points for item in sublist]
manual_track_points_flat_list = [item for sublist in manual_track_points for item in sublist]

### Optical Flow computation 

In [6]:
filename = '/Users/prakash/Dropbox/BobSegMarkData/2018-04-17_fromMark/MAX_20180417_NMY-2_speed-test-01-05_MYOSIN_ALONE_copy.tif'
myosin_channel = imread(filename)
print('Shape of stack:', myosin_channel.shape)
sigma = [0.,2,2]
myosin_channel_smooth = gaussian(myosin_channel,sigma)

Shape of stack: (31, 331, 406)


In [7]:
#Initial parameters for starting the grid search for optimal flow parameters
initial_pyramid_scale = 0.25
initial_pyramid_level = 1
initial_window_size = 3
initial_loops = 1
initial_polygons_n = 1
initial_polygons_sigma = 1
best_relative_offset = 1000000

In [8]:
def parameter_dependent_compute_flow( flowchannel, pyramid_scale, pyramid_levels, window_size, loops, polygons_n, polygons_sigma ):
    '''Computes the Farneback dense flow for the given movie based on passed parameters
    '''
    flows = []
    prvs = flowchannel[0]
    for f in range(flowchannel.shape[0]):
        nxt = flowchannel[f]
        flow = cv2.calcOpticalFlowFarneback(prev=prvs,
                                            next=nxt,
                                            flow=None,
                                            pyr_scale=pyramid_scale, 
                                            levels=pyramid_levels,
                                            winsize=window_size,
                                            iterations=loops,
                                            poly_n=polygons_n, 
                                            poly_sigma=polygons_sigma, 
                                            flags=0)
        flows.append(flow)
        prvs = nxt
        print ('.', end="")
    print (' ...done!')
    return flows

In [9]:
pyramid_scale = initial_pyramid_scale
pyramid_levels = initial_pyramid_level 
window_size = initial_window_size
loops = initial_loops
polygons_n = initial_polygons_n
polygons_sigma = initial_polygons_sigma

while(pyramid_scale<=0.75):
    while(pyramid_levels<=3):
        while(window_size<=7):
            while(loops<=3):
                while(polygons_n<=5):
                    while(polygons_sigma<=2):
                        flows = parameter_dependent_compute_flow( myosin_channel, pyramid_scale, pyramid_levels, window_size, loops, polygons_n, polygons_sigma )
                        flow_x, flow_y = bsu.split_flow_components( flows )
                        avg_flow_x = bsu.flow_average(flow_x, 1)
                        avg_flow_y = bsu.flow_average(flow_y, 1)


                        flow_points = []
                        flow_points_flat_list = []
                        for frame, points in enumerate(center_points):
                            flow_points_per_frame = []
                            for i in range(len(points)):
                                position = bsu.update_pos(points[i], frame, avg_flow_x, avg_flow_y )
                                flow_points_per_frame.append(position)
                                flow_points.append(flow_points_per_frame)
                        
                        flow_points_flat_list = [item for sublist in flow_points for item in sublist]
                        cumulative_relative_offset = 0
                        relative_offset_list = []
                        measure_list = []
    
                        for i in range(len(center_points_flat_list)):
                            offset = bsu.euclid_dist(manual_track_points_flat_list[i], flow_points_flat_list[i])
                            r = bsu.euclid_dist(manual_track_points_flat_list[i], center_points_flat_list[i])
                            relative_offset = offset/r
                            measure = abs(relative_offset-1)
                            relative_offset_list.append(relative_offset)
                            measure_list.append(measure)
                            cumulative_relative_offset = cumulative_relative_offset + measure
                        average_cumulative_relative_offset = cumulative_relative_offset / len(center_points_flat_list)
                       
                        if (average_cumulative_relative_offset<best_relative_offset):
                            best_pyr_scale = pyramid_scale
                            best_levels = pyramid_levels
                            best_win_size = window_size
                            best_iterations = loops
                            best_poly_n = polygons_n
                            best_poly_sigma = polygons_sigma
                            best_relative_offset = average_cumulative_relative_offset
                            
                        polygons_sigma = polygons_sigma + 0.2
                        #print(polygons_sigma)
                        
                        
                    polygons_sigma = initial_polygons_sigma
                    polygons_n = polygons_n+1
                    #print(polygons_n)
                    
                polygons_n = initial_polygons_n
                loops = loops+1
                #print(loops)
                
            loops = initial_loops
            window_size = window_size+2
            #print(window_size)
            
        window_size = initial_window_size
        pyramid_levels = pyramid_levels +1
        #print(pyramid_levels)
        
        
    pyramid_levels =initial_pyramid_level
    pyramid_scale = pyramid_scale+0.1                        

............................... ...done!
1.2
............................... ...done!
1.4
............................... ...done!
1.5999999999999999
............................... ...done!
1.7999999999999998
............................... ...done!
1.9999999999999998
............................... ...done!
2.1999999999999997
2
............................... ...done!
1.2
............................... ...done!
1.4
............................... ...done!
1.5999999999999999
............................... ...done!
1.7999999999999998
............................... ...done!
1.9999999999999998
............................... ...done!
2.1999999999999997
3
............................... ...done!
1.2
............................... ...done!
1.4
............................... ...done!
1.5999999999999999
............................... ...done!
1.7999999999999998
............................... ...done!
1.9999999999999998
............................... ...done!
2.1999999999999997
4
....

............................... ...done!
1.9999999999999998
............................... ...done!
2.1999999999999997
6
3
............................... ...done!
1.2
............................... ...done!
1.4
............................... ...done!
1.5999999999999999
............................... ...done!
1.7999999999999998
............................... ...done!
1.9999999999999998
............................... ...done!
2.1999999999999997
2
............................... ...done!
1.2
............................... ...done!
1.4
............................... ...done!
1.5999999999999999
............................... ...done!
1.7999999999999998
............................... ...done!
1.9999999999999998
............................... ...done!
2.1999999999999997
3
............................... ...done!
1.2
............................... ...done!
1.4
............................... ...done!
1.5999999999999999
............................... ...done!
1.7999999999999998
..

............................... ...done!
1.5999999999999999
............................... ...done!
1.7999999999999998
............................... ...done!
1.9999999999999998
............................... ...done!
2.1999999999999997
6
2
............................... ...done!
1.2
............................... ...done!
1.4
............................... ...done!
1.5999999999999999
............................... ...done!
1.7999999999999998
............................... ...done!
1.9999999999999998
............................... ...done!
2.1999999999999997
2
............................... ...done!
1.2
............................... ...done!
1.4
............................... ...done!
1.5999999999999999
............................... ...done!
1.7999999999999998
............................... ...done!
1.9999999999999998
............................... ...done!
2.1999999999999997
3
............................... ...done!
1.2
............................... ...done!
1.4
..

2.1999999999999997
5
............................... ...done!
1.2
............................... ...done!
1.4
............................... ...done!
1.5999999999999999
............................... ...done!
1.7999999999999998
............................... ...done!
1.9999999999999998
............................... ...done!
2.1999999999999997
6
4
7
............................... ...done!
1.2
............................... ...done!
1.4
............................... ...done!
1.5999999999999999
............................... ...done!
1.7999999999999998
............................... ...done!
1.9999999999999998
............................... ...done!
2.1999999999999997
2
............................... ...done!
1.2
............................... ...done!
1.4
............................... ...done!
1.5999999999999999
............................... ...done!
1.7999999999999998
............................... ...done!
1.9999999999999998
............................... ...done!


............................... ...done!
1.9999999999999998
............................... ...done!
2.1999999999999997
5
............................... ...done!
1.2
............................... ...done!
1.4
............................... ...done!
1.5999999999999999
............................... ...done!
1.7999999999999998
............................... ...done!
1.9999999999999998
............................... ...done!
2.1999999999999997
6
3
............................... ...done!
1.2
............................... ...done!
1.4
............................... ...done!
1.5999999999999999
............................... ...done!
1.7999999999999998
............................... ...done!
1.9999999999999998
............................... ...done!
2.1999999999999997
2
............................... ...done!
1.2
............................... ...done!
1.4
............................... ...done!
1.5999999999999999
............................... ...done!
1.7999999999999998
..

...

............................... ...done!
1.5999999999999999
............................... ...done!
1.7999999999999998
............................... ...done!
1.9999999999999998
............................... ...done!
2.1999999999999997
2
............................... ...done!
1.2
............................... ...done!
1.4
............................... ...done!
1.5999999999999999
............................... ...done!
1.7999999999999998
............................... ...done!
1.9999999999999998
............................... ...done!
2.1999999999999997
3
............................... ...done!
1.2
............................... ...done!
1.4
............................... ...done!
1.5999999999999999
............................... ...done!
1.7999999999999998
............................... ...done!
1.9999999999999998
............................... ...done!
2.1999999999999997
4
............................... ...done!
1.2
............................... ...done!
1.4
....

............................... ...done!
1.2
............................... ...done!
1.4
............................... ...done!
1.5999999999999999
............................... ...done!
1.7999999999999998
............................... ...done!
1.9999999999999998
............................... ...done!
2.1999999999999997
2
............................... ...done!
1.2
............................... ...done!
1.4
............................... ...done!
1.5999999999999999
............................... ...done!
1.7999999999999998
............................... ...done!
1.9999999999999998
............................... ...done!
2.1999999999999997
3
............................... ...done!
1.2
............................... ...done!
1.4
............................... ...done!
1.5999999999999999
............................... ...done!
1.7999999999999998
............................... ...done!
1.9999999999999998
............................... ...done!
2.1999999999999997
4
....

............................... ...done!
1.9999999999999998
............................... ...done!
2.1999999999999997
6
2
............................... ...done!
1.2
............................... ...done!
1.4
............................... ...done!
1.5999999999999999
............................... ...done!
1.7999999999999998
............................... ...done!
1.9999999999999998
............................... ...done!
2.1999999999999997
2
............................... ...done!
1.2
............................... ...done!
1.4
............................... ...done!
1.5999999999999999
............................... ...done!
1.7999999999999998
............................... ...done!
1.9999999999999998
............................... ...done!
2.1999999999999997
3
............................... ...done!
1.2
............................... ...done!
1.4
............................... ...done!
1.5999999999999999
............................... ...done!
1.7999999999999998
..

............................... ...done!
1.5999999999999999
............................... ...done!
1.7999999999999998
............................... ...done!
1.9999999999999998
............................... ...done!
2.1999999999999997
6
4
9
3
............................... ...done!
1.2
............................... ...done!
1.4
............................... ...done!
1.5999999999999999
............................... ...done!
1.7999999999999998
............................... ...done!
1.9999999999999998
............................... ...done!
2.1999999999999997
2
............................... ...done!
1.2
............................... ...done!
1.4
............................... ...done!
1.5999999999999999
............................... ...done!
1.7999999999999998
............................... ...done!
1.9999999999999998
............................... ...done!
2.1999999999999997
3
............................... ...done!
1.2
............................... ...done!
1.

............................... ...done!
2.1999999999999997
5
............................... ...done!
1.2
............................... ...done!
1.4
............................... ...done!
1.5999999999999999
............................... ...done!
1.7999999999999998
............................... ...done!
1.9999999999999998
............................... ...done!
2.1999999999999997
6
3
............................... ...done!
1.2
............................... ...done!
1.4
............................... ...done!
1.5999999999999999
............................... ...done!
1.7999999999999998
............................... ...done!
1.9999999999999998
............................... ...done!
2.1999999999999997
2
............................... ...done!
1.2
............................... ...done!
1.4
............................... ...done!
1.5999999999999999
............................... ...done!
1.7999999999999998
............................... ...done!
1.9999999999999998
..

In [10]:
print("pyr_scale:", best_pyr_scale)
print("levels:", best_levels)
print("win_size:", best_win_size)
print("iterations:", best_iterations)
print("poly_n:", best_poly_n)
print("poly_sigma:",best_poly_sigma )

pyr_scale: 0.5499999999999999
levels: 1
win_size: 3
iterations: 1
poly_n: 5
poly_sigma: 1.9999999999999998


In [11]:
print("Relative offset measure:", best_relative_offset)

Relative offset measure: 28.6445314149
